In [33]:
import argparse
import glob
import io  # for BytesIO
import itertools
# Imports Python standard library logging
import logging
import os
import tarfile
import time
import zipfile
from datetime import datetime

import numpy as np
import pandas as pd
import requests
# W&B
import wandb
# import fastparquet
from google.cloud import storage
from sklearn.ensemble import RandomForestClassifier
# sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

from tqdm import tqdm

import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, LSTM, Dropout, Embedding, Dense, Conv1D, MaxPooling1D
from tensorflow.keras.models import Model


In [34]:
tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [43]:
df = pd.read_csv('/Users/daniel/Synology_Ongoing/HES_Data_Science/AC215/DGAs.csv', index_col=0)
df.drop_duplicates(subset='domains', inplace=True)
df.dropna(inplace=True)
df.info(memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
Index: 29882799 entries, 0 to 30998015
Data columns (total 5 columns):
 #   Column   Dtype  
---  ------   -----  
 0   domains  object 
 1   isdga    int64  
 2   actor    object 
 3   length   int64  
 4   entropy  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 1.3+ GB


In [83]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

def create_lstm_dataset(data_frame, sequence_length=10, batch_size=32, validation_split=0.2):
    """
    Create a TensorFlow Dataset for LSTM classification.

    Parameters:
    - data_frame: Pandas DataFrame containing 'domains' and 'classes.'
    - sequence_length: Length of input sequences for LSTM.
    - batch_size: Batch size for training.
    - validation_split: Fraction of the data to be used for validation.

    Returns:
    - train_dataset: TensorFlow Dataset for training.
    - val_dataset: TensorFlow Dataset for validation.
    - num_classes: Number of unique classes in the dataset.
    """

    # Convert domains and classes to numerical values
    data_frame['classes'] = data_frame['actor']
    unique_classes = data_frame['classes'].unique()
    class_to_index = {cls: idx for idx, cls in enumerate(unique_classes)}
    data_frame['class_index'] = data_frame['classes'].map(class_to_index)

    # Convert each character in the 'domains' column to numeric representation
    char_to_index = {char: idx + 1 for idx, char in enumerate(set(''.join(data_frame['domains'])))}
    data_frame['domain_indices'] = data_frame['domains'].apply(lambda x: np.array([char_to_index[char] for char in x]))

    # Split the dataset into training and validation sets
    train_data, val_data = train_test_split(data_frame, test_size=validation_split) #, stratify=data_frame['class_index'])

    # # Create sequences of input data and corresponding labels
    # def create_sequences(df):
    #     sequences = []
    #     labels = []

    #     for i in range(len(df) - sequence_length):
    #         seq = df['domain_indices'].values[i:i + sequence_length]
    #         label = df['class_index'].values[i + sequence_length]
    #         sequences.append(seq)
    #         labels.append(label)

    #     return sequences, labels

    # train_sequences, train_labels = create_sequences(train_data)
    # val_sequences, val_labels = create_sequences(val_data)

    train_sequences = np.array(train_data['domain_indices'])
    train_labels = np.array(train_data['class_index'])
    val_sequences = np.array(val_data['domain_indices'])
    val_labels = np.array(val_data['class_index'])



    # Convert sequences and labels to TensorFlow Dataset
    print(train_sequences)
    print(train_labels)
    train_dataset = tf.data.Dataset.from_tensor_slices((train_sequences, train_labels))
    val_dataset = tf.data.Dataset.from_tensor_slices((val_sequences, val_labels))

    # Batch and shuffle the datasets
    train_dataset = train_dataset.shuffle(buffer_size=len(train_sequences)).batch(batch_size)
    val_dataset = val_dataset.batch(batch_size)

    # Get the number of unique classes
    num_classes = len(unique_classes)

    return train_dataset, val_dataset, num_classes

# Example usage:
# Assuming you have a DataFrame df with columns 'domains' and 'classes'
# train_dataset, val_dataset, num_classes = create_lstm_dataset(df)


In [84]:
  # Convert domains and classes to numerical values
data_frame =  df_small.copy()
data_frame['classes'] = data_frame['actor']
unique_classes = data_frame['classes'].unique()
class_to_index = {cls: idx for idx, cls in enumerate(unique_classes)}
data_frame['class_index'] = data_frame['classes'].map(class_to_index)

# Convert each character in the 'domains' column to numeric representation
char_to_index = {char: idx + 1 for idx, char in enumerate(set(''.join(data_frame['domains'])))}
data_frame['domain_indices'] = data_frame['domains'].apply(lambda x: [char_to_index[char] for char in x])


In [85]:
data_frame

,domains,isdga,actor,length,entropy,classes,class_index,domain_indices
0,zba93jwfmthdo.com,1,legit,17,3.852169,legit,0,"[31, 20, 36, 14, 35, 8, 27, 13, 22, 21, 11, 7,..."
1,thchsnus.xyz,1,legit,12,3.251629,legit,0,"[21, 11, 30, 11, 1, 25, 5, 1, 19, 6, 17, 31]"
2,viewedhands.com,1,legit,15,3.640224,legit,0,"[15, 33, 2, 27, 2, 7, 11, 36, 25, 7, 1, 19, 30..."
3,4hmjw5q7w9qr.net,1,legit,16,3.750000,legit,0,"[29, 11, 22, 8, 27, 12, 37, 16, 27, 14, 37, 3,..."
4,norduserforum.com,1,legit,17,3.292770,legit,0,"[25, 32, 3, 7, 5, 1, 2, 3, 13, 32, 3, 5, 22, 1..."
5,khhvovnvm.dyndns.org,1,legit,20,3.446439,legit,0,"[9, 11, 11, 15, 32, 15, 25, 15, 22, 19, 7, 17,..."
6,deoyuwxfbxxtteobqgg.com,1,legit,23,3.762267,legit,0,"[7, 2, 32, 17, 5, 27, 6, 13, 20, 6, 6, 21, 21,..."
7,aesuyqwsumkgusaq.org,1,legit,20,3.546439,legit,0,"[36, 2, 1, 5, 17, 37, 27, 1, 5, 22, 9, 18, 5, ..."
8,dgmplybwo.ru,1,legit,12,3.584963,legit,0,"[7, 18, 22, 10, 24, 17, 20, 27, 32, 19, 3, 5]"
9,8xqa87v392w1dq4t.com,1,legit,20,4.121928,legit,0,"[28, 6, 37, 36, 28, 16, 15, 35, 14, 26, 27, 34..."


In [86]:
df_small = df.head(20)

In [87]:
df_small.loc[:10, 'actor'] = 'legit'
df_small.loc[10:, 'actor'] = 'locky'
df_small

,domains,isdga,actor,length,entropy
0,zba93jwfmthdo.com,1,legit,17,3.852169
1,thchsnus.xyz,1,legit,12,3.251629
2,viewedhands.com,1,legit,15,3.640224
3,4hmjw5q7w9qr.net,1,legit,16,3.750000
4,norduserforum.com,1,legit,17,3.292770
5,khhvovnvm.dyndns.org,1,legit,20,3.446439
6,deoyuwxfbxxtteobqgg.com,1,legit,23,3.762267
7,aesuyqwsumkgusaq.org,1,legit,20,3.546439
8,dgmplybwo.ru,1,legit,12,3.584963
9,8xqa87v392w1dq4t.com,1,legit,20,4.121928


In [88]:
MAX_LENGTH = np.max(df_small.domains.apply(len))
print(MAX_LENGTH)

37


In [89]:
create_lstm_dataset(df_small, sequence_length=MAX_LENGTH, batch_size=32, validation_split=0.2)

[array([17,  9, 36,  1, 17, 29, 22, 30, 37, 30, 33, 27, 19, 30, 32, 22])
 array([21, 11, 30, 11,  1, 25,  5,  1, 19,  6, 17, 31])
 array([22, 11, 36, 15, 25, 32, 22,  9, 31, 11, 27,  7,  5, 18,  6, 36, 22,
        19, 25,  2, 21])
 array([ 5, 34,  7, 12, 14, 13, 28, 29, 16,  4, 30, 30, 34,  2, 28, 30,  2,
         7, 12,  2, 16,  4, 28, 36, 36,  7, 36, 16,  7, 16, 36,  2, 29, 16,
        19, 21,  9])
 array([ 7, 18, 22, 10, 24, 17, 20, 27, 32, 19,  3,  5])
 array([10, 23, 29, 35,  2, 30, 23, 36, 29, 14, 29, 34, 28, 20, 16,  4, 23,
        14, 28, 35, 34, 36, 34, 14, 20,  4, 30, 29, 35, 30, 14, 28,  2,  4,
        19, 27,  1])
 array([36,  2,  1,  5, 17, 37, 27,  1,  5, 22,  9, 18,  5,  1, 36, 37, 19,
        32,  3, 18])
 array([ 5, 33, 17, 33,  2,  2, 25, 19, 33, 25, 13, 32])
 array([28,  6, 37, 36, 28, 16, 15, 35, 14, 26, 27, 34,  7, 37, 29, 21, 19,
        30, 32, 22])
 array([25, 32,  3,  7,  5,  1,  2,  3, 13, 32,  3,  5, 22, 19, 30, 32, 22])
 array([13, 31, 22, 25, 20, 20, 21, 37

/var/folders/gb/m76n3l_97j92hstc5f4t0ds00000gn/T/ipykernel_12044/2652206494.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame['classes'] = data_frame['actor']
/var/folders/gb/m76n3l_97j92hstc5f4t0ds00000gn/T/ipykernel_12044/2652206494.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame['class_index'] = data_frame['classes'].map(class_to_index)
/var/folders/gb/m76n3l_97j92hstc5f4t0ds00000gn/T/ipykernel_12044/2652206494.py:29: SettingWithCopyWarning: 
A value is trying to be set on a

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [48]:
def load_tf_dataset(domains):
  vocab = sorted(set(''.join(domains['domain'].to_list())))
  char2idx = {u:i for i, u in enumerate(vocab)}
  idx2char = np.array(vocab)

  lines = []
  for i, line in enumerate(domains.iloc[:, 0]):
    lines.append([char2idx[c] for c in line])
  
  tensor = tf.keras.preprocessing.sequence.pad_sequences(lines, padding='post')

  # Label Encoding
  le = LabelEncoder()
  le.fit(domains.iloc[:, 1])
  
  Y_values_le = le.transform(domains.iloc[:, 1])

  targets = Y_values_le #np.array(domains.iloc[:, 1], dtype=np.int32)

  data = tf.data.Dataset.from_tensor_slices(tensor)
  pred = tf.data.Dataset.from_tensor_slices(targets)
  dataset = tf.data.Dataset.from_tensor_slices((tensor, targets))
  
  return dataset, (char2idx, idx2char)

In [92]:
def load_tf_dataset(domains):
  vocab = sorted(set(''.join(domains['domains'].to_list())))
  char2idx = {u:i for i, u in enumerate(vocab)}
  idx2char = np.array(vocab)

  lines = []
  for i, line in enumerate(domains.iloc[:, 0]):
    lines.append([char2idx[c] for c in line])
  
  tensor = tf.keras.preprocessing.sequence.pad_sequences(lines, padding='pre')


  targets = domains['actor'].astype('category').cat.codes.values


  data = tf.data.Dataset.from_tensor_slices(tensor)
  pred = tf.data.Dataset.from_tensor_slices(targets)
  dataset = tf.data.Dataset.from_tensor_slices((tensor, targets))
  
  return dataset #, (char2idx, idx2char)

In [100]:
for x, y in load_tf_dataset(df_small):
    print(y)

tf.Tensor(0, shape=(), dtype=int8)
tf.Tensor(0, shape=(), dtype=int8)
tf.Tensor(0, shape=(), dtype=int8)
tf.Tensor(0, shape=(), dtype=int8)
tf.Tensor(0, shape=(), dtype=int8)
tf.Tensor(0, shape=(), dtype=int8)
tf.Tensor(0, shape=(), dtype=int8)
tf.Tensor(0, shape=(), dtype=int8)
tf.Tensor(0, shape=(), dtype=int8)
tf.Tensor(0, shape=(), dtype=int8)
tf.Tensor(1, shape=(), dtype=int8)
tf.Tensor(1, shape=(), dtype=int8)
tf.Tensor(1, shape=(), dtype=int8)
tf.Tensor(1, shape=(), dtype=int8)
tf.Tensor(1, shape=(), dtype=int8)
tf.Tensor(1, shape=(), dtype=int8)
tf.Tensor(1, shape=(), dtype=int8)
tf.Tensor(1, shape=(), dtype=int8)
tf.Tensor(1, shape=(), dtype=int8)
tf.Tensor(1, shape=(), dtype=int8)


In [77]:
size_N = 300_000
df = df.reset_index(drop=True)
domains = df[['domains', 'actor']].iloc[:size_N]
domains['domain'] = domains['domains']
domains = domains[['domain', 'actor']]
domains

,domain,actor
0,zba93jwfmthdo.com,shiotob
1,thchsnus.xyz,locky
2,viewedhands.com,nymaim
3,4hmjw5q7w9qr.net,qadars
4,norduserforum.com,legit
...,...,...
299995,mjgwmji3ntga.com,sisron
299996,aawekoacqacqoeqa.org,ramdo
299997,bend-race-choice.com,matsnu
299998,ocmgiqasywemcgsw.org,ramdo


In [78]:
MAX_LENGTH = np.max(domains.domain.apply(len))
print(MAX_LENGTH)

62


In [79]:
dataset, mappings = load_tf_dataset(domains)
char2idx, idx2char = mappings

In [80]:
# Split the dataset into train and test sets
train_size = int(size_N * 0.9)
test_size = len(df) - train_size

train_dataset = dataset.take(train_size).batch(batch_size=1500)
test_dataset = dataset.skip(train_size).batch(batch_size=1500)

In [82]:
first_batch = next(iter(train_dataset.take(1)))
first_component = first_batch[0]
first_component.shape


TensorShape([1500, 62])

In [67]:
# dataset = dataset.batch(1500, drop_remainder=True)
# test_dataset = test_dataset.batch(1500, drop_remainder=True)

In [55]:
def create_model():
  domain_input = Input(shape=(MAX_LENGTH,), dtype='int32', name='domain_input')
  embedding = Embedding(input_dim=39, output_dim=128, input_length=MAX_LENGTH, 
                        batch_input_shape=[1500, None])(domain_input)
  conv = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu', strides=1)(embedding)
  pool = MaxPooling1D(pool_size=2, padding='same')(conv)
  lstm = LSTM(64, return_sequences=False)(pool)
  drop = Dropout(0.5)(lstm)
  output = Dense(1, activation='softmax')(drop)
  model = tf.keras.Model(inputs=domain_input, outputs=output)
  return model

In [56]:
model = create_model()
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [57]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 domain_input (InputLayer)   [(None, 62)]              0         
                                                                 
 embedding_3 (Embedding)     (None, 62, 128)           4992      
                                                                 
 conv1d_3 (Conv1D)           (None, 62, 128)           49280     
                                                                 
 max_pooling1d_3 (MaxPoolin  (None, 31, 128)           0         
 g1D)                                                            
                                                                 
 lstm_3 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                           

In [60]:
EPOCHS = 6 
accuracy = []
losses = []

history = model.fit(train_dataset, validation_data=test_dataset, epochs=EPOCHS)

  # accuracy.append(history.history['accuracy'])
  # losses.append(history.history['loss'])

Epoch 1/6
 35/180 [====>.........................] - ETA: 1:05 - loss: 0.0000e+00 - accuracy: 0.0316

KeyboardInterrupt: 

In [ ]:

# # Initialize a W&B run
# Login into wandb
# wandb.login(key='cb8f805fb5fb66d45b67a3aeb2fa5bb9c5cef739')
# Initialize wandb

model_name = 'DecisionTreeClassifier'
n_estimators = 10 # for RandomForestClassifier

# run = wandb.init(project=GCP_PROJECT, entity='ac2152023_cybersafe', name=model_name, config={"model_name": model_name})

# Define a function to train a decision tree with various parameters
def train_decision_tree(params):
    if model_name == 'RandomForestClassifier':
        model = RandomForestClassifier(**params)
    if model_name == 'DecisionTreeClassifier':
        model = DecisionTreeClassifier(**params)
    model.fit(X_train, y_train)
    return model

# Generate all possible combinations of parameters
param_grid = {
    "criterion" : ['entropy'],
    "max_depth": [4],
    "min_samples_split": [10],
    "min_samples_leaf": [5]
}
all_combinations = itertools.product(*param_grid.values())

# Train a decision tree with each combination of parameters
for combination in all_combinations:

    # Start the timer
    start_time = time.time()

    params = dict(zip(param_grid.keys(),combination))
    if model_name == 'RandomForestClassifier':
        params['n_estimators'] = n_estimators
    
    model = train_decision_tree(params)
    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    accuracy = np.mean(y_pred == y_test)
    
    # Calculate the execution time
    end_time = time.time()
    execution_time = end_time - start_time

    # Log the results to wandb
    # wandb.log({"accuracy": accuracy, "params": params, "execution_time": execution_time})

# Close wandb
# wandb.finish()


logging.debug("Training Job Complete")


In [ ]:
from sklearn import tree
from matplotlib import pyplot as plt

tree.plot_tree(model)
plt.show()